In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install --upgrade flask-ngrok
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=2d9af7c0ec7b4896041ec5122ee89ef9d24f0859222dd23765498568038a8325
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!mkdir -p /root/.ngrok2/

In [ ]:
!echo "authtoken: NGROK_AUTH_TOKEN" > /root/.ngrok2/ngrok.yml

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
model_path = ('./drive/MyDrive/AWS/T5')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_path, legacy=False)


@app.route('/')
def home():
  return "Hello, world!"

@app.route('/model_run', methods=['POST'])
def model_run():
  # Check if the request contains JSON data
  if request.is_json:
      # Extract the "text" field from the JSON data
      text = request.json.get('text', '')
      inputs = tokenizer.encode(text, return_tensors='pt').to(device)
      outputs = model.generate(inputs, max_length=1700, do_sample=True, temperature=0.7)
      decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
      return jsonify({"decoded_output": decoded_output})
  else:
      return jsonify({"error": "Request does not contain valid JSON data"}), 400

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e3ea-35-247-77-151.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
